##雲端硬碟設定指向

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls 'drive'

'My Drive'  'Shared drives'


In [3]:
import os
path = "/content/drive/My Drive/Colab Notebooks/Bert/AML/"
os.chdir(path)

In [4]:
!ls

 albert     NER.ipynb	     分類250.ipynb		     新聞分類訓練.ipynb
 data	    trained_model4  '名字分類(變成姓名學了).ipynb'   新聞分類預測.ipynb
 data.zip   trained_model5  '名字預測(變成姓名學了).ipynb'


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
import torch
from torch.utils.data import TensorDataset
import pickle

In [7]:
import sys 
sys.path.append('.')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '.']

In [8]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForSequenceClassification
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

## 定義函式-快速建立BERT INPUT( tokenizer.build_inputs_with_special_tokens() )

In [9]:
def to_bert_ids(tokenizer,q_input):
    # 將文字輸入轉換成對應的id編號
    #快速建立BERT INPUT
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q_input)))

In [10]:
class DataDic(object):
    def __init__(self, answers):
        self.answers = answers #全部答案(含重複)
        self.answers_norepeat = sorted(list(set(answers))) # 不重複
        self.answers_types = len(self.answers_norepeat) # 總共多少類
        self.ans_list = [] # 用於查找id或是text的list
        self._make_dic() # 製作字典
    
    def _make_dic(self):
        for index_a,a in enumerate(self.answers_norepeat):
            if a != None:
                self.ans_list.append((index_a,a))

    def to_id(self,text):
        for ans_id,ans_text in self.ans_list:
            if text == ans_text:
                return ans_id

    def to_text(self,id):
        for ans_id,ans_text in self.ans_list:
            if id == ans_id:
                return ans_text

    @property
    def types(self):
        return self.answers_types
    
    @property
    def data(self):
        return self.answers

    def __len__(self):
        return len(self.answers)

In [11]:
from torch.utils.data import DataLoader
import torch

In [12]:
!pip install -U ckiptagger[tfgpu,gdown]

     |████████████████████████████████| 411.0MB 40kB/s 
     |████████████████████████████████| 3.8MB 45.5MB/s 
     |████████████████████████████████| 512kB 47.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7ed0bbec34e14f7d844533032962a4161bcecb056477b08a0244e0d9d91750a5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

In [13]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [14]:
def load_data():
    # 使用 GPU：
    #    1. 安裝 tensorflow-gpu (請見安裝說明)
    #    2. 設定 CUDA_VISIBLE_DEVICES 環境變數，例如：os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    #    3. 設定 disable_cuda=False，例如：ws = WS("./data", disable_cuda=False)
    # 使用 CPU：
    global ws, pos,ner
    ws = WS("./data", disable_cuda=False)
    pos = POS("./data", disable_cuda=False)
    ner = NER("./data", disable_cuda=False)

def get_person_str(context):
  
    load_data()

    sentence_list = [context]

    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
        # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )

    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    #print(entity_sentence_list)

    set_entity=set()    
    for i, sentence in enumerate(sentence_list):
        #print()
        #print(f"'{sentence}'")
        #print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        for entity in sorted(entity_sentence_list[i]):
              if entity[2] == 'PERSON':
                #print(entity[3])
                set_entity.add(entity[3].replace(" ", ""))
    str_entity = ','.join(set_entity)
    #print(str_entity)
    return str_entity.split(",")

In [15]:
# load and init
pkl_file = open('trained_model5/data_features.pkl', 'rb')
data_features = pickle.load(pkl_file)
answer_dic = data_features['answer_dic']

In [18]:
model_setting2 = {
    "model_name":"albert", 
    "config_file_path":"trained_model5/config.json", 
    "model_file_path":"trained_model5/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels":2 # 分幾類
}    
model, tokenizer = use_model(**model_setting2)
model.eval()

#
q_inputs = ['自稱房產幽默大師的張仲威，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控，4月28號突然聯繫不上，也沒上課，顧問公司以為他發生危險立刻報案，但人卻已經離開台灣，被害的投資人懷疑，張仲威已經捲款，急忙報警追人。★【男言之癮】男人一看上癮自製youtube頻道，還自稱是房產幽默大師，他是張仲威，不只在網路分享炒房術、如何理財甚至全台跑透透開課，有8千多名學員，這回卻爆出，涉嫌吸金捲款落跑25億。 ▼自稱房產幽默大師 能夠盡早面對。']

for q_input in q_inputs:
    old_q_input=q_input
    if len(tokenizer.tokenize(q_input)) >511 :
      q_input=q_input[0:510]
    bert_ids = to_bert_ids(tokenizer,q_input)
    print('len(bert_ids)',len(bert_ids))
    assert len(bert_ids) <= 512
    input_ids = torch.LongTensor(bert_ids).unsqueeze(0)

    # predict
    outputs = model(input_ids)
    predicts = outputs[:2]
    predicts = predicts[0]
    max_val = torch.max(predicts)
    label = (predicts == max_val).nonzero().numpy()[0][1]
    ans_label = answer_dic.to_text(label)
    
    print(q_input)
    print(ans_label)
    if ans_label =='0':
        print('[]');
    else:
        print(get_person_str(old_q_input)) #['馬勝', '安東尼羅賓', '廖泰宇']

len(bert_ids) 216
自稱房產幽默大師的張仲威，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指控，4月28號突然聯繫不上，也沒上課，顧問公司以為他發生危險立刻報案，但人卻已經離開台灣，被害的投資人懷疑，張仲威已經捲款，急忙報警追人。★【男言之癮】男人一看上癮自製youtube頻道，還自稱是房產幽默大師，他是張仲威，不只在網路分享炒房術、如何理財甚至全台跑透透開課，有8千多名學員，這回卻爆出，涉嫌吸金捲款落跑25億。 ▼自稱房產幽默大師 能夠盡早面對。
1
['張仲威']
